In [1]:
## Sometimes Jupyter notebook doesn't retain your PATH environment variable -- this will mess up a number of things.
## We recommend specifying the environment variable manually here
import os
import ctypes
os.environ["PATH"] = '/home/mattjones/bin:/home/mattjones/.local/bin:/home/mattjones/myapps/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/mattjones/opt/gurobi801/linux64/bin:/home/mattjones/software/bowtie2-2.3.4.2:/home/mattjones/emboss/EMBOSS-6.6.0/emboss'

from cassiopeia.TreeSolver import data_pipeline
from cassiopeia.TreeSolver import post_process_tree as ppt

import pandas as pd 
import numpy as np
import networkx as nx
import pickle as pic

import sys

## Introduction

The minimum input for reconstructing trees with Cassiopeia is a **character matrix** , consisting of indel measurements for **M** *characters* across **N** cells. Here, the term "character" is an abstract term referring to variables containing information about where cells lie in some phylogeny. In our technology, these are individual target sites -- i.e. those sites targeted by Cas9 and subsequently accumulate indels. More generally, these can be nulceotide positions in DNA or even phenotypic observations of an animal. 

We use this abstract data structure because it is general enough for any technology, and does not rely on any assumption other than one can *phase* the observations (i.e. you have the ability to index mutations by where they occurred). 

Here, we begin by describing how one takes an allele table from our PreProcessing Pipeline and converting into a character matrix. To do so, we not only specify an allele table but also a target lineage to reconstruct

In [2]:
alleletable_fp = "lg_output/test.alleleTable.txt"
alleletable = pd.read_csv(alleletable_fp, sep='\t')
target_lg = 4

### Estimate Allele Proportions

As a brief detour, we can estimate the indel priors from the allele tables by measuring the proportion of intBCs that report a given indel. The idea behind this is that we'd like to subset the data into groups of independent sets (i.e. those that are not related to one another at all) as this will eliminate the possiblity of "double counting" indels because they were inherited. Inevitably, we will underestimate the propensity of a given indel to occur if it happened many times in the same location (as we choose to assume this only occurs by inheritance); however, the more independent groups we have, the greater our ability to estimate priors.

The function ``get_indel_props`` will do this and takes the following arguments:

- **at**: Allele Table Dataframe
- **group_var**: A list of variables by which to subset the data into independent groups (Default = ['intBC'])

This will return a dataframe with three columns -- the indel identity, the proportion of groups it appeared in, and the frequency of this indel (an estimate of the prior probability).

In [3]:
allele_props = data_pipeline.get_indel_props(alleletable)

Counting unique alleles: 100%|██████████| 124/124 [00:00<00:00, 9964.44it/s]


### Create character matrix from a lineage group

We now aggregate our observations from the allele table into a character matrix -- a matrix whose dimensions are `n x m`, where we have `n` cells and `m` characters (in our case, this is the number of total target sites, 3 times the number of integrations). We can use the ``alleletable_to_character_matrix`` function here, which takes the following parameters:

- **at**: alleletable (subsetted for a particular lineage group) 
- **out_fp**: the output file path for the resulting character matrix (Default = None). This value is only necessary when ``write = True``.
- **mutation_map**: the indel priors, calculated as above with ``get_indel_props``. If nothing is provided, no prior probabilities will be used for later tree reconstructions.
- **no_context**: Use indels without context (default = False).
- **write**: Write to file.

This function will return (or write) three files: 
- the character matrix
- the indel proprotions dictionary, specifying the mutation probabilities for each character to a particular state
- dictionary translating a state-character pair to an observed allele. 

If write is True, the last two files are saved as pickle files, and only written if a mutation_map is provided. Else, the dictionaries are returned. 

In [4]:
lg = alleletable[alleletable["lineageGrp"] == target_lg]

data_pipeline.alleletable_to_character_matrix(lg, "test_lg4_character_matrix.txt", mutation_map=allele_props, write=True)

Processing characters: 100%|██████████| 45/45 [00:00<00:00, 1300.82it/s]


### Reconstruct a lineage

You can now reconstruct lineages using the `reconstruct-lineage` command, which takes in many different options depending on which of many lineage solvers you'd like to use. We provide Neighbor-Joining, Camin-Sokal (implemented with PHYLIP), greedy, Steiner-Tree/ILP, and Hybrid algorithms. Use the `-h` flag to see all possible parameters.

The output of this procedure will be a Cassiopeia_Tree object, which is essentially a wrapper for a networkx object representing the Tree, and a newick string. This object also contains several other utilities functions which can be found on our documentation website: https://cassiopeia-lineage.readthedocs.io/en/latest/Cassiopeia.TreeSolver.html

### Post-Processing a Tree

Here we'll work with a tree named `test_lg4_tree.pkl`, and we assume it was constructed with the greedy method. In this case, there are two things that need to be done:

1. Map terminal character states to the cell identifiers
2. Add "Redundant" leaves to the terminal leaves. This is necessary because if not ever cell represents a unique state, then the final tree would only be over a subset of the cells originally in the character matrix. This is known as the Post-Processing Tree Step.

In the event of post-processing a tree constructed with the hybrid or ilp methods, we only perform step 2. 

Instead of using this in a python environment, there is also a command-line tool provided for this step: `post-process-tree`. Use the `-h` flag to see options and usage.

#### Mapping Terminal states

In [26]:
g = nx.read_gpickle("test_lg4_tree.pkl")
cm = pd.read_csv("test_lg4_character_matrix.txt", sep='\t', index_col = 0)

#### Post-Process Tree & Add Redundant Leaves

In [27]:
g = g.post_process_tree(cm = cm)

# rewrite newick file and save it in the object
g.newick = data_pipeline.convert_network_to_newick_format(g)

removing character strings from sample names: 100%|██████████| 1048/1048 [00:00<00:00, 581509.54it/s]


In [28]:
## now save final tre
out_fp = "test_lg4_tree.processed.pkl"

pic.dump(g, open(out_fp, 'wb'))